In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,803 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,062 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 4,201 kB in 2s (

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-06-19 04:58:00--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2023-06-19 04:58:01 (10.6 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [4]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [5]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz'
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"), inferSchema=True, sep='\t')
df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [6]:
# Get the number of rows in the DataFrame.
row_count = df.count()
print("Number of rows: ", row_count)


Number of rows:  4366916


# Transform the Data

## Create the "review_id_table".

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date

# Create a SparkSession
spark = SparkSession.builder.appName("Create_review_id_df").getOrCreate()

# Select the required columns and transform the data types
review_id_df = df.select(
    df["review_id"],
    df["customer_id"],
    to_date(df["review_date"], "yyyy-MM-dd").alias("review_date"),
    df["product_id"],
    df["star_rating"]
)

# Show the review_id_df DataFrame
review_id_df.show()


+--------------+-----------+-----------+----------+-----------+
|     review_id|customer_id|review_date|product_id|star_rating|
+--------------+-----------+-----------+----------+-----------+
|R3P2HIOQCIN5ZU|   18069663| 2015-08-31|B000XB31C0|          1|
|R12VVR0WH5Q24V|   16251825| 2015-08-31|B00CFYZH5W|          5|
| RNCCKB6TV5EEF|   20381037| 2015-08-31|B00S8JNN3Q|          4|
|R2NZXYIVCGB13W|     108364| 2015-08-31|B00XFBPOQG|          5|
|R2EQ1TG9IT3OEQ|   45449350| 2015-08-31|B00SW64Y9W|          3|
|R1WXA9JSC2H1U1|   19324665| 2015-08-31|B011F9E6LI|          5|
|R12ENYLFGGNWRV|   50073594| 2015-08-31|B00HAUP1OI|          5|
|R2R07E5PNXEUO5|   21706057| 2015-08-31|B00L1RKOKW|          4|
|R27BA52AKWMWN3|   13708216| 2015-08-31|B005WA9MSE|          5|
| RLF8DOID2KD5O|   40542649| 2015-08-31|B00BEYQI5C|          3|
|R369CEXHXHC6NQ|   13409446| 2015-08-31|B00EYAFTCQ|          5|
|R171PPIJXFONVI|    9451727| 2015-08-31|B00I0QHY32|          5|
|R2JDNM8F2P06FU|     193731| 2015-08-31|

## Create the "products" Table

In [8]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("Create products_df").getOrCreate()

# Assuming you have already read the data into a DataFrame called "df"
# Drop duplicates based on "product_id" and "product_title" columns
products_df = df.dropDuplicates(["product_id", "product_title"])

# Show the products_df DataFrame
products_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   30125982|R2Q87GEKBBYM14|0456846999|     759453678|Blogger Eyewear S...|           Shoes|          5|            0|          0|   N|                Y|          Five Stars|Beautiful shades,...| 2014-09-19|
|         US|   44781845|R2Q2LNW0MRQUGW|1428597565|     974716845|Anne Michelle Lon...|           Shoes|          5|    

## Create the "customers" Table

In [9]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product.
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("Create customers_df").getOrCreate()

# Assuming you have already read the data into a DataFrame called "df"
# Group the data by "customer_id" and count the number of reviews for each customer
customers_df = df.groupBy("customer_id").count()

# Show the customers_df DataFrame
customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   16121565|    5|
|   48146680|    1|
|   11062912|    1|
|   51451778|    1|
|   27920838|    1|
|    4919528|    3|
|   47802851|    2|
|   42560427|    1|
|   29467780|    1|
|   49703087|   12|
|   38456994|    4|
|     557836|    1|
|   43783459|    1|
|   23607725|    1|
|   45632184|    1|
|   19979069|    2|
|   28380959|    1|
|   46563381|    1|
|   21628739|    3|
|   23042837|    2|
+-----------+-----+
only showing top 20 rows



## Create the "vine_table".

In [10]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns.
# Create a SparkSession
spark = SparkSession.builder.appName("Create vine_df").getOrCreate()

# Assuming you have already read the data into a DataFrame called "df"
# Select the desired columns to create the vine_df DataFrame
vine_df = df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")

# Show the vine_df DataFrame
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|
|R12VVR0WH5Q24V|          5|            0|          0|   N|
| RNCCKB6TV5EEF|          4|            0|          0|   N|
|R2NZXYIVCGB13W|          5|            0|          6|   N|
|R2EQ1TG9IT3OEQ|          3|            0|          0|   N|
|R1WXA9JSC2H1U1|          5|            1|          1|   N|
|R12ENYLFGGNWRV|          5|            1|          1|   N|
|R2R07E5PNXEUO5|          4|            0|          0|   N|
|R27BA52AKWMWN3|          5|            0|          0|   N|
| RLF8DOID2KD5O|          3|            0|          0|   N|
|R369CEXHXHC6NQ|          5|            0|          0|   N|
|R171PPIJXFONVI|          5|            0|          0|   N|
|R2JDNM8F2P06FU|          4|            1|          1|   N|
|R2W977FO4M97XT|          5|            

# Load

In [11]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [49]:
# Write the DataFrame to the RDS table
reviews_df.write.jdbc(url=jdbc_url, table=table, mode=mode, properties=config)

Py4JJavaError: ignored

In [33]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='shoes_products_df', mode=mode, properties=config)

Py4JJavaError: ignored

In [34]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_products_df', mode=mode, properties=config)

Py4JJavaError: ignored

In [35]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_products_df', mode=mode, properties=config)

Py4JJavaError: ignored